In [3]:
#importing the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [4]:
#load the claims data

data=pd.read_csv(r"final_data.csv",header=0)
data_bk=data.copy()
data.head()

,Location,MinTemp,Rainfall,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Temp3pm,RainToday,RainTomorrow,Season
0,0.458788,0.535620,0.021898,0.771236,0.527778,0.692830,0.932527,0.512821,0.545455,0.666667,0.212121,0.253102,0.502439,0,0,0.4
1,0.458788,0.377309,0.000000,0.957365,0.527778,1.000000,0.476489,0.102564,0.500000,0.356322,0.242424,0.325062,0.563415,0,0,0.4
2,0.458788,0.522427,0.000000,0.616632,0.555556,0.692830,0.476489,0.487179,0.590909,0.287356,0.292929,0.250620,0.536585,0,0,0.4
3,0.458788,0.424802,0.000000,0.281836,0.250000,0.106936,0.014389,0.282051,0.204545,0.367816,0.151515,0.498759,0.617073,0,0,0.4
4,0.458788,0.643799,0.036496,0.771236,0.486111,0.191606,1.000000,0.179487,0.454545,0.793103,0.323232,0.330025,0.695122,0,0,0.4


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138472 entries, 0 to 138471
Data columns (total 16 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Location       138472 non-null  float64
 1   MinTemp        138472 non-null  float64
 2   Rainfall       138472 non-null  float64
 3   WindGustDir    138472 non-null  float64
 4   WindGustSpeed  138472 non-null  float64
 5   WindDir9am     138472 non-null  float64
 6   WindDir3pm     138472 non-null  float64
 7   WindSpeed9am   138472 non-null  float64
 8   WindSpeed3pm   138472 non-null  float64
 9   Humidity9am    138472 non-null  float64
 10  Humidity3pm    138472 non-null  float64
 11  Pressure9am    138472 non-null  float64
 12  Temp3pm        138472 non-null  float64
 13  RainToday      138472 non-null  int64  
 14  RainTomorrow   138472 non-null  int64  
 15  Season         138472 non-null  float64
dtypes: float64(14), int64(2)
memory usage: 16.9 MB


In [6]:
indepvar=[]
for col in data.columns:
    if col!='RainTomorrow':
        indepvar.append(col)
depvar='RainTomorrow'
x=data[indepvar]
y=data[depvar]

In [7]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=42)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((96930, 15), (41542, 15), (96930,), (41542,))

Boosting algorithms on RandomForest

In [5]:
import pandas as pd

from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score, matthews_corrcoef, jaccard_score, roc_auc_score, log_loss, cohen_kappa_score, average_precision_score, brier_score_loss, fowlkes_mallows_score
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import GradientBoostingClassifier
# Assuming df_resampled is your DataFrame
# Split the data into features (X) and target variable (y)


# Define base models
base_models = {
    'Random Forest': RandomForestClassifier()
}

# Define boosting models
boosting_models = {
    'CatBoost': CatBoostClassifier(),
    'XGBoost': XGBClassifier(),
    'AdaBoost': AdaBoostClassifier(),
    'LightGBM': LGBMClassifier(),
    'GradientBoostingClassifier':GradientBoostingClassifier()
}

def calculate_metrics(y_true, y_pred, y_pred_prob):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

    metrics = {
        'Accuracy': accuracy_score(y_true, y_pred),
        'Precision': precision_score(y_true, y_pred),
        'Recall': recall_score(y_true, y_pred),
        'F1 Score': f1_score(y_true, y_pred),
        'Specificity': tn / (tn + fp),
        'Balanced Accuracy': balanced_accuracy_score(y_true, y_pred),
        'MCC': matthews_corrcoef(y_true, y_pred),
        'ROC AUC': roc_auc_score(y_true, y_pred_prob),
        'Jaccard Index': jaccard_score(y_true, y_pred),
        'Logloss': log_loss(y_true, y_pred_prob),
        'Cohen\'s Kappa': cohen_kappa_score(y_true, y_pred),
        'Average Precision': average_precision_score(y_true, y_pred_prob),
        'Brier Loss': brier_score_loss(y_true, y_pred),
        'Fowlkes-Mallows Index': fowlkes_mallows_score(y_true, y_pred),
        'Gini Coefficient': 2 * roc_auc_score(y_true, y_pred_prob) - 1,
        'Balanced Classification Rate (BCR)': (recall_score(y_true, y_pred) + tn / (tn + fp)) / 2
    }

    return metrics

# Create an empty DataFrame to store the results
results_df = pd.DataFrame(columns=['Model', 'Base Model'] + list(calculate_metrics(y_test, y_test, y_test).keys()))

# Loop through each base model
for base_model_name, base_model in base_models.items():

    # Create the boosted model
    for boosting_model_name, boosting_model in boosting_models.items():
        boosted_model = boosting_model  # Remove the function call

        # Train the model on the training dataset
        boosted_model.fit(x_train, y_train)

        # Make predictions on the test set
        y_pred = boosted_model.predict(x_test)
        y_pred_prob = boosted_model.predict_proba(x_test)[:, 1]

        # Calculate metrics
        metrics_dict = calculate_metrics(y_test, y_pred, y_pred_prob)

        # Append results to the DataFrame
        results_df = results_df.append({
            'Model': f'Boosted {boosting_model_name} + {base_model_name} Model',
            'Base Model': base_model_name,
            **metrics_dict
        }, ignore_index=True)

Learning rate set to 0.072636
0:	learn: 0.6403447	total: 148ms	remaining: 2m 28s
1:	learn: 0.5975929	total: 167ms	remaining: 1m 23s
2:	learn: 0.5646785	total: 186ms	remaining: 1m 1s
3:	learn: 0.5340974	total: 204ms	remaining: 50.8s
4:	learn: 0.5093633	total: 221ms	remaining: 44.1s
5:	learn: 0.4886305	total: 238ms	remaining: 39.5s
6:	learn: 0.4714460	total: 253ms	remaining: 36s
7:	learn: 0.4574492	total: 277ms	remaining: 34.4s
8:	learn: 0.4447593	total: 291ms	remaining: 32s
9:	learn: 0.4345372	total: 304ms	remaining: 30.1s
10:	learn: 0.4264568	total: 317ms	remaining: 28.5s
11:	learn: 0.4191974	total: 332ms	remaining: 27.3s
12:	learn: 0.4130876	total: 347ms	remaining: 26.4s
13:	learn: 0.4070560	total: 361ms	remaining: 25.4s
14:	learn: 0.4026098	total: 375ms	remaining: 24.7s
15:	learn: 0.3984348	total: 389ms	remaining: 23.9s
16:	learn: 0.3949630	total: 404ms	remaining: 23.4s
17:	learn: 0.3912241	total: 420ms	remaining: 22.9s
18:	learn: 0.3885513	total: 434ms	remaining: 22.4s
19:	learn: 0.

162:	learn: 0.3395101	total: 2.93s	remaining: 15.1s
163:	learn: 0.3393797	total: 2.95s	remaining: 15s
164:	learn: 0.3392189	total: 2.97s	remaining: 15s
165:	learn: 0.3391038	total: 2.98s	remaining: 15s
166:	learn: 0.3389338	total: 3s	remaining: 15s
167:	learn: 0.3388074	total: 3.02s	remaining: 14.9s
168:	learn: 0.3387128	total: 3.04s	remaining: 14.9s
169:	learn: 0.3385876	total: 3.05s	remaining: 14.9s
170:	learn: 0.3384824	total: 3.07s	remaining: 14.9s
171:	learn: 0.3383494	total: 3.09s	remaining: 14.9s
172:	learn: 0.3382196	total: 3.1s	remaining: 14.8s
173:	learn: 0.3381421	total: 3.12s	remaining: 14.8s
174:	learn: 0.3380092	total: 3.14s	remaining: 14.8s
175:	learn: 0.3378512	total: 3.15s	remaining: 14.8s
176:	learn: 0.3377170	total: 3.17s	remaining: 14.7s
177:	learn: 0.3376102	total: 3.19s	remaining: 14.7s
178:	learn: 0.3374454	total: 3.2s	remaining: 14.7s
179:	learn: 0.3372966	total: 3.22s	remaining: 14.7s
180:	learn: 0.3371764	total: 3.24s	remaining: 14.7s
181:	learn: 0.3370380	tot

324:	learn: 0.3229765	total: 5.7s	remaining: 11.8s
325:	learn: 0.3228757	total: 5.71s	remaining: 11.8s
326:	learn: 0.3227806	total: 5.73s	remaining: 11.8s
327:	learn: 0.3226347	total: 5.75s	remaining: 11.8s
328:	learn: 0.3225249	total: 5.76s	remaining: 11.8s
329:	learn: 0.3224423	total: 5.78s	remaining: 11.7s
330:	learn: 0.3223943	total: 5.8s	remaining: 11.7s
331:	learn: 0.3223398	total: 5.81s	remaining: 11.7s
332:	learn: 0.3222675	total: 5.83s	remaining: 11.7s
333:	learn: 0.3221963	total: 5.85s	remaining: 11.7s
334:	learn: 0.3221386	total: 5.86s	remaining: 11.6s
335:	learn: 0.3220570	total: 5.88s	remaining: 11.6s
336:	learn: 0.3219911	total: 5.9s	remaining: 11.6s
337:	learn: 0.3218996	total: 5.91s	remaining: 11.6s
338:	learn: 0.3218546	total: 5.93s	remaining: 11.6s
339:	learn: 0.3217682	total: 5.95s	remaining: 11.5s
340:	learn: 0.3216404	total: 5.96s	remaining: 11.5s
341:	learn: 0.3215689	total: 5.98s	remaining: 11.5s
342:	learn: 0.3215059	total: 6s	remaining: 11.5s
343:	learn: 0.3214

491:	learn: 0.3114751	total: 8.45s	remaining: 8.73s
492:	learn: 0.3114129	total: 8.47s	remaining: 8.71s
493:	learn: 0.3113468	total: 8.49s	remaining: 8.7s
494:	learn: 0.3113012	total: 8.51s	remaining: 8.68s
495:	learn: 0.3112414	total: 8.52s	remaining: 8.66s
496:	learn: 0.3111286	total: 8.54s	remaining: 8.64s
497:	learn: 0.3110848	total: 8.56s	remaining: 8.63s
498:	learn: 0.3110275	total: 8.57s	remaining: 8.61s
499:	learn: 0.3109736	total: 8.59s	remaining: 8.59s
500:	learn: 0.3109248	total: 8.6s	remaining: 8.57s
501:	learn: 0.3108850	total: 8.62s	remaining: 8.55s
502:	learn: 0.3108084	total: 8.64s	remaining: 8.53s
503:	learn: 0.3107262	total: 8.65s	remaining: 8.52s
504:	learn: 0.3106691	total: 8.67s	remaining: 8.5s
505:	learn: 0.3106037	total: 8.69s	remaining: 8.48s
506:	learn: 0.3105431	total: 8.7s	remaining: 8.46s
507:	learn: 0.3104952	total: 8.72s	remaining: 8.45s
508:	learn: 0.3104443	total: 8.74s	remaining: 8.43s
509:	learn: 0.3103428	total: 8.76s	remaining: 8.41s
510:	learn: 0.31

659:	learn: 0.3018142	total: 11.2s	remaining: 5.78s
660:	learn: 0.3017550	total: 11.2s	remaining: 5.76s
661:	learn: 0.3016634	total: 11.2s	remaining: 5.74s
662:	learn: 0.3016121	total: 11.3s	remaining: 5.72s
663:	learn: 0.3015746	total: 11.3s	remaining: 5.7s
664:	learn: 0.3015239	total: 11.3s	remaining: 5.68s
665:	learn: 0.3014719	total: 11.3s	remaining: 5.67s
666:	learn: 0.3014234	total: 11.3s	remaining: 5.65s
667:	learn: 0.3013660	total: 11.3s	remaining: 5.63s
668:	learn: 0.3013136	total: 11.3s	remaining: 5.61s
669:	learn: 0.3012576	total: 11.4s	remaining: 5.59s
670:	learn: 0.3012025	total: 11.4s	remaining: 5.57s
671:	learn: 0.3011516	total: 11.4s	remaining: 5.55s
672:	learn: 0.3010992	total: 11.4s	remaining: 5.54s
673:	learn: 0.3010324	total: 11.4s	remaining: 5.52s
674:	learn: 0.3009908	total: 11.4s	remaining: 5.5s
675:	learn: 0.3009464	total: 11.4s	remaining: 5.48s
676:	learn: 0.3008795	total: 11.5s	remaining: 5.47s
677:	learn: 0.3008267	total: 11.5s	remaining: 5.45s
678:	learn: 0.

829:	learn: 0.2933830	total: 13.9s	remaining: 2.85s
830:	learn: 0.2933295	total: 13.9s	remaining: 2.84s
831:	learn: 0.2932885	total: 14s	remaining: 2.82s
832:	learn: 0.2932435	total: 14s	remaining: 2.8s
833:	learn: 0.2932149	total: 14s	remaining: 2.79s
834:	learn: 0.2931805	total: 14s	remaining: 2.77s
835:	learn: 0.2931447	total: 14s	remaining: 2.75s
836:	learn: 0.2931039	total: 14s	remaining: 2.73s
837:	learn: 0.2930561	total: 14.1s	remaining: 2.72s
838:	learn: 0.2930282	total: 14.1s	remaining: 2.7s
839:	learn: 0.2929893	total: 14.1s	remaining: 2.68s
840:	learn: 0.2929470	total: 14.1s	remaining: 2.67s
841:	learn: 0.2929143	total: 14.1s	remaining: 2.65s
842:	learn: 0.2928787	total: 14.1s	remaining: 2.63s
843:	learn: 0.2928311	total: 14.1s	remaining: 2.62s
844:	learn: 0.2927765	total: 14.2s	remaining: 2.6s
845:	learn: 0.2927300	total: 14.2s	remaining: 2.58s
846:	learn: 0.2926889	total: 14.2s	remaining: 2.56s
847:	learn: 0.2926405	total: 14.2s	remaining: 2.55s
848:	learn: 0.2925957	total

998:	learn: 0.2858612	total: 16.5s	remaining: 16.5ms
999:	learn: 0.2858249	total: 16.5s	remaining: 0us


C:\Users\Maha\AppData\Local\Temp\ipykernel_26120\1403201541.py:75: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({
C:\Users\Maha\AppData\Local\Temp\ipykernel_26120\1403201541.py:75: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({
C:\Users\Maha\AppData\Local\Temp\ipykernel_26120\1403201541.py:75: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({


[LightGBM] [Info] Number of positive: 19999, number of negative: 76931
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004661 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1304
[LightGBM] [Info] Number of data points in the train set: 96930, number of used features: 15
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.206324 -> initscore=-1.347227
[LightGBM] [Info] Start training from score -1.347227


C:\Users\Maha\AppData\Local\Temp\ipykernel_26120\1403201541.py:75: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({
C:\Users\Maha\AppData\Local\Temp\ipykernel_26120\1403201541.py:75: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({


In [6]:
results_df

,Model,Base Model,Accuracy,Precision,Recall,F1 Score,Specificity,Balanced Accuracy,MCC,ROC AUC,Jaccard Index,Logloss,Cohen's Kappa,Average Precision,Brier Loss,Fowlkes-Mallows Index,Gini Coefficient,Balanced Classification Rate (BCR)
0,Boosted CatBoost + Random Forest Model,Random Forest,0.860238,0.730722,0.491747,0.587876,0.953927,0.722837,0.522009,0.876804,0.416306,0.328195,0.507569,0.695686,0.139762,0.834688,0.753608,0.722837
1,Boosted XGBoost + Random Forest Model,Random Forest,0.857205,0.710611,0.498634,0.586043,0.948371,0.723503,0.514474,0.872204,0.414470,0.334624,0.502948,0.684008,0.142795,0.830397,0.744408,0.723503
2,Boosted AdaBoost + Random Forest Model,Random Forest,0.846324,0.706886,0.413253,0.521583,0.956432,0.684843,0.459830,0.840614,0.352798,0.675757,0.437439,0.630734,0.153676,0.825288,0.681229,0.684843
3,Boosted LightGBM + Random Forest Model,Random Forest,0.856170,0.719097,0.476665,0.573306,0.952658,0.714662,0.506074,0.868351,0.401842,0.337996,0.491054,0.678852,0.143830,0.831089,0.736703,0.714662
4,Boosted GradientBoostingClassifier + Random Fo...,Random Forest,0.851211,0.719867,0.435459,0.542656,0.956916,0.696187,0.480915,0.852819,0.372360,0.353824,0.460165,0.653456,0.148789,0.828958,0.705637,0.696187
